# SIADS 516: Homework 1
Version 1.0.20200221.1
### Dr. Chris Teplovs, School of Information, University of Michigan
<small><a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-nc-sa/4.0/88x31.png" /></a>This work is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/">Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International License</a>.

### Our first mrjob script

Recall the following example from the lectures:

Note the use of the magic command ```%%file```.  You can use this to write the contents of a cell out to a file, which is what we need to do to use mrjob:

In [9]:
%%file word_count.py
from mrjob.job import MRJob
import re

class MRWordFrequencyCount(MRJob):

  ### input: self, in_key, in_value
  def mapper(self, _, line):
    yield "chars", len(line)
    yield "words", len(line.split())
    yield "lines", 1

  ### input: self, in_key from mapper, in_value from mapper
  def reducer(self, key, values):
    yield key, sum(values)
if __name__ == "__main__":
    MRWordFrequencyCount.run()

Overwriting word_count.py


### <font color="magenta">Q1: Explain what each of the yield statements in the above script do.  Provide a list of what the first few iterations through the mapper() step would yield if the script was run against the ```data/gutenberg/short.t1.txt``` file.

The first yield statement gives the count of characters. The second yield statements gives the count of words. The third yield statement gives the count of line(s). The last yield statment gives the summarised values for characters, words, and lines, throughout the text file. 

first 5 iterations:
"chars"	69
"words"	12
"lines"	1

"chars"	0
"words"	0
"lines"	1

"chars"	64
"words"	14
"lines"	1

"chars"	68
"words"	12
"lines"	1

"chars"	69
"words"	12
"lines"	1

Now let's look at the output of running the script against that same file:

In [10]:
!python word_count.py data/gutenberg/short.t1.txt

No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory /tmp/word_count.jovyan.20210308.040651.802760
Running step 1 of 1...
job output is in /tmp/word_count.jovyan.20210308.040651.802760/output
Streaming final output from /tmp/word_count.jovyan.20210308.040651.802760/output...
"chars"	10653
"words"	1822
"lines"	200
Removing temp directory /tmp/word_count.jovyan.20210308.040651.802760...


### <font color="magenta">Q2.  Repeat the above cell using the the works of William Shakespeare text file (data/gutenberg/t8.shakespeare.txt).  Provide an interpretation of the output (don't overthink this -- just demonstrate that you can find the relevant information in the output).</font>

In [11]:
!python word_count.py data/gutenberg/t8.shakespeare.txt

No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory /tmp/word_count.jovyan.20210308.040717.136616
Running step 1 of 1...
job output is in /tmp/word_count.jovyan.20210308.040717.136616/output
Streaming final output from /tmp/word_count.jovyan.20210308.040717.136616/output...
"chars"	5333743
"words"	901325
"lines"	124456
Removing temp directory /tmp/word_count.jovyan.20210308.040717.136616...


there are 124456 lines in the text file, with 901325 words and 5333743 characters.

Insert your interpretation here.

### Now let's look at a slightly more complicated example:

In [52]:
%%file most_used_word.py
from mrjob.job import MRJob
from mrjob.step import MRStep
import re

WORD_RE = re.compile(r"[\w']+") # any whitespace or apostrophe, used to split lines below


class MRMostUsedWord(MRJob):
    STOPWORDS = {'i', 'we', 'ourselves', 'hers', 'between', 'yourself', 'but', 'again', 'there', 'about', 'once', 'during', 'out', 'very', 'having', 'with', 'they', 'own', 'an', 'be', 'some', 'for', 'do', 'its', 'yours', 'such', 'into', 'of', 'most', 'itself', 'other', 'off', 'is', 's', 'am', 'or', 'who', 'as', 'from', 'him', 'each', 'the', 'themselves', 'until', 'below', 'are', 'we', 'these', 'your', 'his', 'through', 'don', 'nor', 'me', 'were', 'her', 'more', 'himself', 'this', 'down', 'should', 'our', 'their', 'while', 'above', 'both', 'up', 'to', 'ours', 'had', 'she', 'all', 'no', 'when', 'at', 'any', 'before', 'them', 'same', 'and', 'been', 'have', 'in', 'will', 'on', 'does', 'yourselves', 'then', 'that', 'because', 'what', 'over', 'why', 'so', 'can', 'did', 'not', 'now', 'under', 'he', 'you', 'herself', 'has', 'just', 'where', 'too', 'only', 'myself', 'which', 'those', 'i', 'after', 'few', 'whom', 't', 'being', 'if', 'theirs', 'my', 'against', 'a', 'by', 'doing', 'it', 'how', 'further', 'was', 'here', 'than'}
    def steps(self):
        return [
            MRStep(mapper=self.mapper_get_words,
                   combiner=self.combiner_count_words,
                   reducer=self.reducer_count_words),
            MRStep(reducer=self.reducer_find_max_word)
        ]

    def mapper_get_words(self, _, line):
        # yield each word in the line
        for word in WORD_RE.findall(line):
            if word.lower() not in self.STOPWORDS:
                yield (word.lower(), 1)

    def combiner_count_words(self, word, counts):
        # optimization: sum the words we've seen so far
        yield (word, sum(counts))

    def reducer_count_words(self, word, counts):
        # send all (num_occurrences, word) pairs to the same reducer.
        # num_occurrences is used so we can easily use Python's max() function.
        yield None, (sum(counts), word)

    # discard the key; it is just None
    def reducer_find_max_word(self, _, word_count_pairs):
        # each item of word_count_pairs is (count, word),
        # so yielding one results in key=counts, value=word
        yield max(word_count_pairs)



if __name__ == '__main__':
    import time
    start = time.time()
    MRMostUsedWord.run()
    end = time.time()
    print(end - start)

Overwriting most_used_word.py


### <font color="magenta">Q3: Explain what the yield statements in the  above script do.  Provide a list of what the first few iterations through the steps would yield.

(Assuming there's only the first line in the file, according to Sean from Slack discussion: https://umich-mads.slack.com/archives/G01PV0P9CQG/p1614894175075100)

The yield statement under mapper_get_words gives every word in the text file except the ones in the stopwords list, in lower case format, with '1' as the count of the word itself:

"project"	1
"gutenberg's"	1
"year"	1
"2889"	1
"jules"	1
"verne"	1
"michel"	1
"verne"	1


The yield statment under combiner_count_words takes the results above and gives word-counts pairs which sums the count of the same words occur on the same line:

"2889"	1
"gutenberg's"	1
"jules"	1
"michel"	1
"project"	1
"verne"	2
"year"	1

The yield statement under reducer_count_words takes the word-counts pairs above and gives the word-counts pairs of the same words across the text file.

null	[1, "2889"]
null	[1, "michel"]
null	[1, "project"]
null	[1, "jules"]
null	[1, "year"]
null	[2, "verne"]
null	[1, "gutenberg's"]

The yield statment under reducer_find_max_word finds the word-counts pair from the aobve pairs.

2	"verne"

Now run the file against data/gutenberg/short.t1.txt.

In [55]:
!python most_used_word.py data/gutenberg/t8.shakespeare.txt

No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory /tmp/most_used_word.jovyan.20210308.060219.287286
Running step 1 of 2...
Running step 2 of 2...
job output is in /tmp/most_used_word.jovyan.20210308.060219.287286/output
Streaming final output from /tmp/most_used_word.jovyan.20210308.060219.287286/output...
5479	"thou"
Removing temp directory /tmp/most_used_word.jovyan.20210308.060219.287286...
7.887237787246704


### <font color="magenta">Q4: Run the above script on the Shakespeare text file.  What answer do you get?</font>

max word:
5479	"thou"

### <font color="magenta">Q5: What is the impact of removing the combiner from the above code in terms of efficiency?  What does that suggest?</font>

In [49]:
%%file most_used_word2.py
from mrjob.job import MRJob
from mrjob.step import MRStep
import re

WORD_RE = re.compile(r"[\w']+") # any whitespace or apostrophe, used to split lines below


class MRMostUsedWord(MRJob):
    STOPWORDS = {'i', 'we', 'ourselves', 'hers', 'between', 'yourself', 'but', 'again', 'there', 'about', 'once', 'during', 'out', 'very', 'having', 'with', 'they', 'own', 'an', 'be', 'some', 'for', 'do', 'its', 'yours', 'such', 'into', 'of', 'most', 'itself', 'other', 'off', 'is', 's', 'am', 'or', 'who', 'as', 'from', 'him', 'each', 'the', 'themselves', 'until', 'below', 'are', 'we', 'these', 'your', 'his', 'through', 'don', 'nor', 'me', 'were', 'her', 'more', 'himself', 'this', 'down', 'should', 'our', 'their', 'while', 'above', 'both', 'up', 'to', 'ours', 'had', 'she', 'all', 'no', 'when', 'at', 'any', 'before', 'them', 'same', 'and', 'been', 'have', 'in', 'will', 'on', 'does', 'yourselves', 'then', 'that', 'because', 'what', 'over', 'why', 'so', 'can', 'did', 'not', 'now', 'under', 'he', 'you', 'herself', 'has', 'just', 'where', 'too', 'only', 'myself', 'which', 'those', 'i', 'after', 'few', 'whom', 't', 'being', 'if', 'theirs', 'my', 'against', 'a', 'by', 'doing', 'it', 'how', 'further', 'was', 'here', 'than'}
    def steps(self):
        return [
            MRStep(mapper=self.mapper_get_words,
                  
                   reducer=self.reducer_count_words),
            MRStep(reducer=self.reducer_find_max_word)
        ]

    def mapper_get_words(self, _, line):
        # yield each word in the line
        for word in WORD_RE.findall(line):
            if word.lower() not in self.STOPWORDS:
                yield (word.lower(), 1)

   

    def reducer_count_words(self, word, counts):
        # send all (num_occurrences, word) pairs to the same reducer.
        # num_occurrences is used so we can easily use Python's max() function.
        yield None, (sum(counts), word)

    # discard the key; it is just None
    def reducer_find_max_word(self, _, word_count_pairs):
        # each item of word_count_pairs is (count, word),
        # so yielding one results in key=counts, value=word
        yield max(word_count_pairs)



if __name__ == '__main__':
    import time
    start = time.time()
    MRMostUsedWord.run()
    end = time.time()
    print(end - start)

Writing most_used_word2.py


In [51]:
!python most_used_word2.py data/gutenberg/t8.shakespeare.txt

No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory /tmp/most_used_word2.jovyan.20210308.060125.083941
Running step 1 of 2...
Running step 2 of 2...
job output is in /tmp/most_used_word2.jovyan.20210308.060125.083941/output
Streaming final output from /tmp/most_used_word2.jovyan.20210308.060125.083941/output...
5479	"thou"
Removing temp directory /tmp/most_used_word2.jovyan.20210308.060125.083941...
6.140726566314697


From the example of Shakespeare text file above, there is a decrease in the execution time of the python code (from ~8 seconds to ~6 seconds). it suggests that more computation power would be consumed if we added an extra mrjob function, as there is extra handling for each iteration. It should suggest linear time complexity (O(n)).

### <font color="magenta">Q6: Write an mrjob script that finds the 10 words that have the most syllables from the t5.churchill.txt file.  Interpret your results.</font>

In [50]:
%%file top10_most_syllables.py
from mrjob.job import MRJob
from mrjob.step import MRStep
import syllables
import re

WORD_RE = re.compile(r"[\w']+") 
    
class MRMostSyllables(MRJob):

    def steps(self):
        return [
            MRStep(mapper=self.mapper_get_words),
            MRStep(reducer=self.reducer),
            MRStep(reducer=self.reducer_find_top_word)
        ]

     
    def mapper_get_words (self, _, line):
        # yield each word in the line
        for word in WORD_RE.findall(line):
            word = word.lower()
            syllables_count = syllables.estimate(word)
            yield (syllables_count, word), 1
        
        
    def reducer(self, key, values):
        yield None,(key, sum(values))        
    
    def reducer_find_top_word(self, _, key_val_pair):
        l =sorted(key_val_pair, key = lambda x: (x[0][0], x[1]), reverse = True)
        for i in l[:10]:
            yield i
    

if __name__ == '__main__':
    import time
    start = time.time()
    MRMostSyllables.run()
    end = time.time()
    print(end - start)
    

Overwriting top10_most_syllables.py


In [51]:
!python top10_most_syllables.py data/gutenberg/t5.churchill.txt

No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory /tmp/top10_most_syllables.jovyan.20210308.145823.925381
Running step 1 of 3...
Running step 2 of 3...
Running step 3 of 3...
job output is in /tmp/top10_most_syllables.jovyan.20210308.145823.925381/output
Streaming final output from /tmp/top10_most_syllables.jovyan.20210308.145823.925381/output...
[8, "incommunicability"]	1
[8, "overcapitalization"]	1
[7, "apologetically"]	29
[7, "characteristically"]	22
[7, "incompatibility"]	6
[7, "inaccessibility"]	4
[7, "imperturbability"]	3
[7, "inconsiderately"]	3
[7, "uncommunicative"]	3
[7, "disproportionately"]	2
Removing temp directory /tmp/top10_most_syllables.jovyan.20210308.145823.925381...
75.42973279953003


from the results we can interpret that the top 10 most syllable words in t5.churchill.txt file are:
1. incommunicability, 8 syllables, occured 1 time in the text file
2. overcapitalization, 8 syllables, occured 1 time in the text file
3. apologetically, 7 syllables, occured 29 times in the text file
4. characteristically, 7 syllables, occured 22 times in the text file
5. incompatibility, 7 syllables, occured 6 times in the text file
6. inaccessibility, 7 syllables, occured 4 times in the text file
7. imperturbability, 7 syllables, occured 3 times in the text file
8. inconsiderately, 7 syllables, occured 3 times in the text file
9. uncommunicative, 7 syllables, occured 3 times in the text file
10. disproportionately, 7 syllables, occured 2 times in the text file

because there are many words with 7 syllables, we use word frequency as a secondary factor to determine the top10 list.